In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA

# SOM class
class TSOM:

    def __init__(self,
                 D = 1,
                 n = 3,
                 resolution = 50,
                 sigma_max = 1.0,
                 sigma_min = 0.05,
                 tau = 50.0
                 ):
        self.D = D
        self.n = n
        self.resolution = resolution
        self.sigma_max = sigma_max
        self.sigma_min = sigma_min
        self.tau = tau

        self.k_star_1 = np.zeros((10, 10))
        self.k_star_2 = np.zeros((10, 10))
        self.zeta_1 = None
        self.zeta_2 = None
        self.z_1 = np.zeros((10, self.resolution))
        self.z_2 = np.zeros((10, self.resolution))
        self.h_1 = None
        self.h_2 = None
        self.sigma = None
        self.u_1 = None
        self.u_2 = None
        self.y = None

    def fit(self, data, t):
        #self.initialize(data)
        self.__e_step(data)
        self.__m_step(data, t)

    def initialize(self, data):
        self.zeta_1 = create_zeta(self.resolution, self.D)
        self.zeta_2 = create_zeta(self.resolution, self.D)
        self.u_1 = data
        self.u_2 = data
        self.y = np.random.rand(10, 10, 3)

    def __e_step(self, data):
        n = 0
        for n1 in range(10):
            self.k_star_1[n1] = np.argmin(cdist(self.u_1[n1, :, :], self.y[n1, :, :]), axis=1)
        for n2 in range(10):
            self.k_star_2[n2] = np.argmin(cdist(self.u_2[:, n2, :], self.y[:, n2, :]), axis=1)
        for n1 in range(10):
            self.z_1[n1] = self.zeta_1[self.k_star_1[n1], :]
        for n2 in range(10):
            self.z_2[n2] = self.zeta_2[self.k_star_2[n2], :]

    def __m_step(self, data, t):
        self.h_1 = np.exp(-0.5 * cdist(self.zeta_1, self.z_1) / (self.__sigma(t) ** 2))
        self.h_2 = np.exp(-0.5 * cdist(self.zeta_2, self.z_2) / (self.__sigma(t) ** 2))
        self.u_1 = np.sum(self.h_1[None, :, :, None] * data[:, :, None, :] / np.sum(self.h_2, axis=0), axis=1)
        self.u_2 = np.sum(self.h_2[None, :, :, None] * data[:, :, None, :] / np.sum(self.h_1, axis=0), axis=0)
        self.y = np.dot(self.h, data) / np.sum(self.h, axis=1)[:, None]

    def __sigma(self, t):
        self.sigma = self.sigma_max + (self.sigma_min - self.sigma_max) * (t / self.tau)
        if self.sigma < self.sigma_min:
            self.sigma = self.sigma_min
        return self.sigma

def create_zeta(resolution, D):
    zeta = np.array([np.linspace(-1, 1, resolution)] * 10)
    zeta = np.reshape(zeta, (10, resolution, D))
    return zeta


In [258]:
np.zeros((2,2))

array([[0., 0.],
       [0., 0.]])

In [259]:
np.random.rand(2,2)

array([[0.7213801 , 0.00994376],
       [0.42890525, 0.26351366]])

In [216]:
import numpy as np
from scipy.spatial.distance import cdist

#load data
data = np.loadtxt("datatsom.txt")
data = np.reshape(data, (10, 10, 3))

resolution = 50
D = 1
def create_zeta(resolution, D):
    zeta = np.array(np.linspace(-1, 1, resolution))
#     zeta = np.reshape(zeta, (resolution, D))
    return zeta

In [253]:
sigma_max = 1.0
sigma_min = 0.05
tau = 50.0
def sigma(t):
    sigma = sigma_max + (sigma_min - sigma_max) * (t / tau)
    if sigma < sigma_min:
        sigma = sigma_min
    return sigma

In [245]:
zeta_1 = create_zeta(resolution, D)
u_1 = data
y = np.random.rand(10, 10, 3)
z_1 = np.zeros(10)

In [246]:
k_star_1 = np.argmin(np.sum(np.power(u_1[:, None, :, :] - y[None, :, :, :], 2), axis=(2,3)), axis=1)

In [247]:
k_star_1

array([0, 8, 8, 8, 8, 8, 8, 8, 0, 0], dtype=int64)

In [248]:
z_1 = zeta_1[k_star_1]

In [249]:
zeta_1

array([-1.        , -0.95918367, -0.91836735, -0.87755102, -0.83673469,
       -0.79591837, -0.75510204, -0.71428571, -0.67346939, -0.63265306,
       -0.59183673, -0.55102041, -0.51020408, -0.46938776, -0.42857143,
       -0.3877551 , -0.34693878, -0.30612245, -0.26530612, -0.2244898 ,
       -0.18367347, -0.14285714, -0.10204082, -0.06122449, -0.02040816,
        0.02040816,  0.06122449,  0.10204082,  0.14285714,  0.18367347,
        0.2244898 ,  0.26530612,  0.30612245,  0.34693878,  0.3877551 ,
        0.42857143,  0.46938776,  0.51020408,  0.55102041,  0.59183673,
        0.63265306,  0.67346939,  0.71428571,  0.75510204,  0.79591837,
        0.83673469,  0.87755102,  0.91836735,  0.95918367,  1.        ])

In [250]:
z_1

array([-1.        , -0.67346939, -0.67346939, -0.67346939, -0.67346939,
       -0.67346939, -0.67346939, -0.67346939, -1.        , -1.        ])

In [255]:
r_1 = np.exp(-0.5 * np.power(zeta_1[:,None]-z_1[None,:], 2) / (sigma(1)**2))

In [256]:
r_1

array([[1.        , 0.94611025, 0.94611025, 0.94611025, 0.94611025,
        0.94611025, 0.94611025, 0.94611025, 1.        , 1.        ],
       [0.99913481, 0.95847414, 0.95847414, 0.95847414, 0.95847414,
        0.95847414, 0.95847414, 0.95847414, 0.99913481, 0.99913481],
       [0.99654373, 0.96932013, 0.96932013, 0.96932013, 0.96932013,
        0.96932013, 0.96932013, 0.96932013, 0.99654373, 0.99654373],
       [0.99224018, 0.97859331, 0.97859331, 0.97859331, 0.97859331,
        0.97859331, 0.97859331, 0.97859331, 0.99224018, 0.99224018],
       [0.98624641, 0.98624641, 0.98624641, 0.98624641, 0.98624641,
        0.98624641, 0.98624641, 0.98624641, 0.98624641, 0.98624641],
       [0.97859331, 0.99224018, 0.99224018, 0.99224018, 0.99224018,
        0.99224018, 0.99224018, 0.99224018, 0.97859331, 0.97859331],
       [0.96932013, 0.99654373, 0.99654373, 0.99654373, 0.99654373,
        0.99654373, 0.99654373, 0.99654373, 0.96932013, 0.96932013],
       [0.95847414, 0.99913481, 0.9991348

In [257]:
np.size(r_1, axis=0)

50

In [179]:
k_star_1

array([[4, 4, 4, 4, 4, 4, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
       [3, 3, 3, 3, 3, 3, 3, 5, 5, 5],
       [8, 8, 8, 8, 8, 8, 4, 4, 4, 4],
       [8, 8, 8, 8, 8, 8, 5, 5, 5, 5],
       [6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
       [8, 8, 8, 8, 8, 8, 8, 8, 2, 2],
       [4, 4, 4, 4, 4, 4, 4, 5, 0, 0],
       [2, 2, 2, 2, 2, 7, 7, 3, 8, 8],
       [1, 1, 1, 7, 7, 7, 7, 7, 5, 5]])

In [180]:
k_star_1[0]

array([4, 4, 4, 4, 4, 4, 6, 6, 6, 6])

In [181]:
zeta_1

array([-1.        , -0.95918367, -0.91836735, -0.87755102, -0.83673469,
       -0.79591837, -0.75510204, -0.71428571, -0.67346939, -0.63265306,
       -0.59183673, -0.55102041, -0.51020408, -0.46938776, -0.42857143,
       -0.3877551 , -0.34693878, -0.30612245, -0.26530612, -0.2244898 ,
       -0.18367347, -0.14285714, -0.10204082, -0.06122449, -0.02040816,
        0.02040816,  0.06122449,  0.10204082,  0.14285714,  0.18367347,
        0.2244898 ,  0.26530612,  0.30612245,  0.34693878,  0.3877551 ,
        0.42857143,  0.46938776,  0.51020408,  0.55102041,  0.59183673,
        0.63265306,  0.67346939,  0.71428571,  0.75510204,  0.79591837,
        0.83673469,  0.87755102,  0.91836735,  0.95918367,  1.        ])

In [192]:
zeta_1[k_star_1]

array([[-0.83673469, -0.83673469, -0.83673469, -0.83673469, -0.83673469,
        -0.83673469, -0.75510204, -0.75510204, -0.75510204, -0.75510204],
       [-0.75510204, -0.75510204, -0.75510204, -0.75510204, -0.75510204,
        -0.75510204, -0.75510204, -0.75510204, -0.75510204, -0.75510204],
       [-0.87755102, -0.87755102, -0.87755102, -0.87755102, -0.87755102,
        -0.87755102, -0.87755102, -0.79591837, -0.79591837, -0.79591837],
       [-0.67346939, -0.67346939, -0.67346939, -0.67346939, -0.67346939,
        -0.67346939, -0.83673469, -0.83673469, -0.83673469, -0.83673469],
       [-0.67346939, -0.67346939, -0.67346939, -0.67346939, -0.67346939,
        -0.67346939, -0.79591837, -0.79591837, -0.79591837, -0.79591837],
       [-0.75510204, -0.75510204, -0.75510204, -0.75510204, -0.75510204,
        -0.75510204, -0.75510204, -0.75510204, -0.75510204, -0.75510204],
       [-0.67346939, -0.67346939, -0.67346939, -0.67346939, -0.67346939,
        -0.67346939, -0.67346939, -0.67346939

In [183]:
z_1

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [184]:
zeta_1[9]

-0.6326530612244898

In [188]:
for n1 in range(10):
#     print(n1)
    for n2 in range(10):
#         print(n2)
        z_1[n1][n2] = zeta_1[k_star_1[n1][n2]]
#         zeta_1[k_star_1[n1][n2]]
#         nn = k_star_1[n1][n2]
#         print(nn)
#         zeta_1[nn]

In [189]:
z_1

array([[-0.83673469, -0.83673469, -0.83673469, -0.83673469, -0.83673469,
        -0.83673469, -0.75510204, -0.75510204, -0.75510204, -0.75510204],
       [-0.75510204, -0.75510204, -0.75510204, -0.75510204, -0.75510204,
        -0.75510204, -0.75510204, -0.75510204, -0.75510204, -0.75510204],
       [-0.87755102, -0.87755102, -0.87755102, -0.87755102, -0.87755102,
        -0.87755102, -0.87755102, -0.79591837, -0.79591837, -0.79591837],
       [-0.67346939, -0.67346939, -0.67346939, -0.67346939, -0.67346939,
        -0.67346939, -0.83673469, -0.83673469, -0.83673469, -0.83673469],
       [-0.67346939, -0.67346939, -0.67346939, -0.67346939, -0.67346939,
        -0.67346939, -0.79591837, -0.79591837, -0.79591837, -0.79591837],
       [-0.75510204, -0.75510204, -0.75510204, -0.75510204, -0.75510204,
        -0.75510204, -0.75510204, -0.75510204, -0.75510204, -0.75510204],
       [-0.67346939, -0.67346939, -0.67346939, -0.67346939, -0.67346939,
        -0.67346939, -0.67346939, -0.67346939

In [28]:
k_star_2 = np.argmin(cdist(u_1[:, None, :, :], y[None, :, :, :]), axis=1)

ValueError: XA must be a 2-dimensional array.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from SOM_T import TSOM

#load data
data = np.loadtxt("datatsom.txt")
data = np.reshape(data, (10, 10, 3))

#learn
som = TSOM()
som.initialize(data)
for t in range(50):
    som.fit(data, t)


fig = plt.figure()
ax = fig.gca(projection='3d')
ax.scatter(data[:, 0], data[:, 1], data[:, 2])
ax.plot(som.y[:, 0], som.y[:, 1], som.y[:, 2])

plt.show()

ModuleNotFoundError: No module named 'SOM_T'